# Light Guide Panel Generator

Light Guide Panels are the (literal) backbone of LCD screen illumination.
They're the diffusion secret sauce behind uniformly backlit LCD displays.
When illuminated from the edge, they illuminate outwards through the pattern etched on the surface of the arylic.

It turns out that design software for this sort of thing [already exists](https://www.febees.com/index.html). But why pay \$1500 when we can write a crude version that will do the job?

In [26]:
#%matplotlib inline
import matplotlib.pyplot as plt
import drawSvg as draw
import numpy as np
from scipy.optimize import curve_fit
import scipy.interpolate as interpolate
from ipywidgets import interactive
from IPython.display import SVG, display, HTML
import base64


# Extra fn for rescaling displayed SVGs
_html_template='<img width="{}" src="data:image/svg+xml;base64,{}" >'

# def svg_to_fixed_width_html_image(svg, width="100%"):
#     text = _html_template.format(width, base64.b64encode(svg))
#     return HTML(text)

def svg_to_fixed_width_html_image(svg, width="50%"):
    b64 = base64.b64encode(svg).decode("utf8")
    text = f'<img width="{width}" src="data:image/svg+xml;base64,{b64}" >'
    return HTML(text) 

In [85]:
HALF_WIDTH_MM = 8.5*25.4 / 2
HEIGHT_MM = 14 * 25.4
NOMINAL_DOTS_PER_MM = 2.5

We can create a spline function with handles that can help us model dot density as a function of distance from the light source.

In [86]:
def spline_density_gen(y0, x1, y1, x2, y2, x3, y3, y4):
    x = np.array([0, x1, x2, x3, HALF_WIDTH_MM])  # x axis control points 
    y = np.array([y0, y1, y2, y3, y4])  # y control points.
    t, c, k = interpolate.splrep(x, y, s=0, k=2)
    spline = interpolate.BSpline(t, c, k, extrapolate=False)
    # plot:
    plt.figure(figsize=(10, 5))
    #plt.axes().set_aspect(100)#('equal')
    plt.plot(x, y, 'bo', label='Control Points')  # Plot the control points.
    N = 100  # Plot resolution.
    xx = np.linspace(x.min(), x.max(), N)
    plt.plot(xx, spline(xx), 'r', label='BSpline')
    plt.plot(xx, [1/NOMINAL_DOTS_PER_MM]*len(xx))
    plt.grid()
    plt.legend(loc='best')
    plt.title("Half Length Illumination Density")
    plt.ylabel("dot density [dots/mm]")
    plt.xlabel("distance from center [mm]")
    plt.tight_layout()
    #plt.ylim(0,1)
    plt.show()
    return t,c,k


i_sdf = interactive(spline_density_gen,
                y0=(0, 1/NOMINAL_DOTS_PER_MM*2, 0.01), 
                x1=(0, HALF_WIDTH_MM,0.25),
                y1=(0, 1/NOMINAL_DOTS_PER_MM*2, 0.01),
                x2=(0,HALF_WIDTH_MM,0.25),
                y2=(0, 1/NOMINAL_DOTS_PER_MM*2, 0.01),
                x3=(0,HALF_WIDTH_MM,0.25),
                y3=(0, 1/NOMINAL_DOTS_PER_MM*2, 0.01),
                y4=(0, 1/NOMINAL_DOTS_PER_MM*2, 0.01))
display(i_sdf)

interactive(children=(FloatSlider(value=0.4, description='y0', max=0.8, step=0.01), FloatSlider(value=53.75, d…

In [117]:
# Create the spline function from the density parameters above.
rho = interpolate.BSpline(*i_sdf.result, extrapolate=False)

def dotspace(x0, xf, endpoint=False):
    """return an array of dot locations from x0 to xf spaced 
        according to the dot density profile."""
    dots_x = []
    x = x0
    while x < xf:
        dots_x.append(x)
        dots_per_mm_x = rho(x)
        x += 1/dots_per_mm_x  # Step forward mm-per-dot
    return np.array(dots_x)

In [118]:

Y = np.arange(0, HEIGHT_MM, NOMINAL_DOTS_PER_MM)[:-1]
X = dotspace(0, HALF_WIDTH_MM)
xx, yy = np.meshgrid(X, Y)

yoffset = (Y[1] - Y[0])/2.
xoffset = (X[1] - X[0])/2
Y2 = np.arange(yoffset, HEIGHT_MM+yoffset, NOMINAL_DOTS_PER_MM)[:-1]
X2 = dotspace(xoffset, HALF_WIDTH_MM)
xx2, yy2 = np.meshgrid(X2,Y2)

# Move non-illuminated sides off the top/bottom edge.
yy += NOMINAL_DOTS_PER_MM/2.
yy2 += NOMINAL_DOTS_PER_MM/2.

# TODO: we need to renormalize the output since it has been stretched beyond the width of the original img.

# Preview Plot for sanity checking:
# fig = plt.figure(figsize=(15,15))
# ax = fig.add_subplot(111)
# ax.set_aspect('equal')
# ax.plot(xx, yy, ls="None", marker=".", color="blue")
# ax.plot(xx2, yy2, ls="None", marker=".", color="green")
# ax.plot(-xx, yy, ls="None", marker=".", color="blue")
# ax.plot(-xx2, yy2, ls="None", marker=".", color="green")
# plt.tight_layout()
# plt.show()

Draw the SVG from the above parameters.

In [119]:
# Create an SVG replacing the two meshgrids from above with actual circles.
dot_diameter = 0.75 # mm

offset = (10, 10)
d = draw.Drawing(2*HALF_WIDTH_MM + 2*offset[0], HEIGHT_MM + 2*offset[0],
                 origin=(-HALF_WIDTH_MM - offset[0], 0),
                 displayInline=False)
r = draw.Rectangle(-HALF_WIDTH_MM,0, 2*HALF_WIDTH_MM, HEIGHT_MM, fill='#ffffff',
                   stroke='#ff00ff', stroke_width="1")
d.append(r)
# Produce the circle pattern.
for xx, yy in [[xx, yy], [xx2, yy2], [-xx, yy], [-xx2, yy2]]:
    for x,y in zip(xx.flatten(), yy.flatten()):
        c = draw.Circle(x, y, r=dot_diameter/2., stroke_width='0.1', stroke='black',
                        fill_opacity=1.0)#, id='circle')
        d.append(c)
d.saveSvg('example.svg')

In [120]:
#display(SVG(url='https://upload.wikimedia.org/wikipedia/commons/f/f6/People_%28example%29.svg'))
svg_to_fixed_width_html_image(SVG('example.svg').data.encode('ascii'), width="100%")